In [108]:
# import required python libraries
#import geopandas as gpd  # for geo-data structure
import numpy as np  # for some operation better done with numpy
import matplotlib.pyplot as plt  # for creating plots
#import shapely.geometry  # for geometry data crunching
#from shapely.geometry import Point  # specifically for point geometry data manipulation
#import keplergl
#from keplergl import KeplerGl
import datacompy
import pandas as pd
crs = {'init': 'epsg:4326'}
#from shapely.geometry import Point
import mpl_toolkits


In [72]:
# bring in the 3 to 4 different coverage dataset from FB, GSMA, ITU and Ericsson as well as the master school_geolocation

In [106]:
RW_f = pd.read_csv('./data/RW_FB_data_20210608.csv',low_memory=False,encoding='utf-8')
RW_itu = pd.read_csv('./data/coverage_data_itu.csv',low_memory=False,encoding='utf-8')
RW_e = pd.read_csv('./data/Rwanda_Ericsson_07_06_2021.csv',low_memory=False,encoding='utf-8')
RW_master = pd.read_csv('./data/rwanda_school_geolocation_master.csv',low_memory=False,encoding='utf-8')

In [83]:
# some colunm names standardization

In [114]:
RW_e.rename(columns={'SCHOOL_CODE':'school_id', 'SCHOOL_NAME':'school_name'}, inplace=True)
RW_itu.rename(columns={'giga_schoo': 'giga_school_id','source_sch':'school_id', 'name':'school_name'}, inplace=True)

In [116]:
# for facebook
cross_profile_fb_master = datacompy.Compare(
RW_master,
RW_f,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Facebook'
)
#for ericsson
cross_profile_e_master = datacompy.Compare(
RW_master,
RW_e,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Ericsson'
)
# for ITU
cross_profile_itu_master = datacompy.Compare(
RW_master,
RW_itu,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='ITU'
)

print(cross_profile_fb_master.report())
print(cross_profile_e_master.report())
print(cross_profile_itu_master.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0    Master       20  4093
1  Facebook       26  4195

Column Summary
--------------

Number of columns in common: 12
Number of columns in Master but not in Facebook: 8
Number of columns in Facebook but not in Master: 14

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 4,091
Number of rows in Master but not in Facebook: 2
Number of rows in Facebook but not in Master: 104

Number of rows with some compared columns unequal: 8
Number of rows with all compared columns equal: 4,083

Column Comparison
-----------------

Number of columns compared with some values unequal: 1
Number of columns compared with all values equal: 11
Total number of values which compare unequal: 8

Columns with Unequal Values or Types
------------------------------------

        Column Master dtype Face

In [74]:
# they usually don't come with "coverage" colunms, we create it at source level using the following codes

In [75]:
RW_e['coverage'] = np.where((RW_e['2G_coverage']== 'None') & (RW_e['3G_coverage']== 'None')& (RW_e['4G_coverage']== 'None'), 'NO', 'YES')
RW_f['coverage'] = np.where((RW_f['percent_2g'].isnull()) & (RW_f['percent_3g'].isnull())& (RW_f['percent_4g'].isnull()), 'NO', 'YES')
RW_f['coverage'] = np.where((RW_f['percent_2g']<=0) & (RW_f['percent_3g']<=0) & (RW_f['percent_4g']<=0), 'NO', 'YES')
RW_itu['coverage'] = np.where((RW_itu['2G'].isnull()) & (RW_itu['3G'].isnull())& (RW_itu['4G'].isnull()), 'NO', 'YES')

In [76]:
# create the 2G, 3G and 4G coverage columns with binary values at source level using the following codes

In [77]:
RW_f['2G_coverage'] = np.where((RW_f['percent_2g'] > 0), 'YES', 'NO')
RW_f['3G_coverage'] = np.where((RW_f['percent_3g'] > 0), 'YES', 'NO')
RW_f['4G_coverage'] = np.where((RW_f['percent_4g'] > 0), 'YES', 'NO')

In [78]:
RW_e.rename(columns={'2G_coverage':'2G_coverage_quality', '3G_coverage':'3G_coverage_quality', '4G_coverage': '4G_coverage_quality'}, inplace=True)

In [79]:
RW_e['2G_coverage'] = np.where((RW_e['2G_coverage_quality'] != 'None'), 'YES', 'NO')
RW_e['3G_coverage'] = np.where((RW_e['3G_coverage_quality'] != 'None'), 'YES', 'NO')
RW_e['4G_coverage'] = np.where((RW_e['4G_coverage_quality'] != 'None'), 'YES', 'NO')
RW_itu['2G_coverage'] = np.where((RW_itu['2G'] >=1), 'YES', 'NO')
RW_itu['3G_coverage'] = np.where((RW_itu['3G'] == 1), 'YES', 'NO')
RW_itu['4G_coverage'] = np.where((RW_itu['4G'] ==1), 'YES', 'NO')

In [80]:
# they usually don't come with "coverage_type" colunms, we create it at source level using the following function and codes

In [81]:
def coverage_type(df):
    if df['4G_coverage'] == 'YES':
        return "4G"
    elif df['3G_coverage'] == 'YES':
        return "3G"
    elif df['2G_coverage'] == 'YES':
        return "2G"
    else:
        return "None"

In [82]:
RW_f['coverage_type'] = RW_f.apply((lambda x: coverage_type(x)),axis=1)
RW_e['coverage_type'] = RW_e.apply((lambda x: coverage_type(x)),axis=1)
RW_itu['coverage_type'] = RW_itu.apply((lambda x: coverage_type(x)),axis=1)

In [85]:
# isolating the needed colunms from the rest

In [86]:
features = ['giga_school_id','coverage','coverage_type','2G_coverage','3G_coverage','4G_coverage'] 

In [87]:
RW_fc = RW_f[features]
RW_ec = RW_e[features]
RW_it = RW_itu[features]

In [88]:
# more sanity checks for nan on giga_school_id

In [89]:
RW_fc = RW_fc[RW_fc['giga_school_id'].notna()]
RW_ec = RW_ec[RW_ec['giga_school_id'].notna()]
RW_it = RW_it[RW_it['giga_school_id'].notna()]

In [90]:
# visually comparing the sources for matches and mismatches across the columns

In [91]:
df_all = pd.concat([RW_fc.set_index('giga_school_id'), RW_ec.set_index('giga_school_id'),RW_it.set_index('giga_school_id')], 
                   axis='columns', keys=['Facebook', 'Ericsson', 'ITU'])
df_all.head()

Facebook                                                    \
               coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
giga_school_id                                                              
G_RWA_00112         YES            4G         YES         YES         YES   
G_RWA_00116         YES            4G         YES         YES         YES   
G_RWA_00258         YES            4G         YES         YES         YES   
G_RWA_00551         YES            4G         YES         YES         YES   
G_RWA_00901         YES            4G         YES         YES         YES   

               Ericsson                                                    \
               coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
giga_school_id                                                              
G_RWA_00112         YES            4G         YES         YES         YES   
G_RWA_00116         YES            4G         YES         YES         YES   
G_RWA_00258         YES            4G         YES         YES         YES   
G_RWA_00551         YES            4G         YES         YES         YES   
G_RWA_00901         YES            4G         YES         YES         YES   

                    ITU                                                    
               coverage coverage_type 2G_coverage 3G_coverage 4G_coverage  
giga_school_id                                                             
G_RWA_00112         YES            4G         YES         YES         YES  
G_RWA_00116         YES            4G         YES          NO         YES  
G_RWA_00258         YES            4G         YES          NO         YES  
G_RWA_00551         YES            4G         YES         YES         YES  
G_RWA_00901         YES            4G         YES          NO         YES

In [92]:
df_final = df_all.swaplevel(axis='columns')[RW_fc.columns[1:]]
df_final.head()

coverage               coverage_type              2G_coverage  \
               Facebook Ericsson  ITU      Facebook Ericsson ITU    Facebook   
giga_school_id                                                                 
G_RWA_00112         YES      YES  YES            4G       4G  4G         YES   
G_RWA_00116         YES      YES  YES            4G       4G  4G         YES   
G_RWA_00258         YES      YES  YES            4G       4G  4G         YES   
G_RWA_00551         YES      YES  YES            4G       4G  4G         YES   
G_RWA_00901         YES      YES  YES            4G       4G  4G         YES   

                             3G_coverage               4G_coverage           \
               Ericsson  ITU    Facebook Ericsson  ITU    Facebook Ericsson   
giga_school_id                                                                
G_RWA_00112         YES  YES         YES      YES  YES         YES      YES   
G_RWA_00116         YES  YES         YES      YES   NO         YES      YES   
G_RWA_00258         YES  YES         YES      YES   NO         YES      YES   
G_RWA_00551         YES  YES         YES      YES  YES         YES      YES   
G_RWA_00901         YES  YES         YES      YES   NO         YES      YES   

                     
                ITU  
giga_school_id       
G_RWA_00112     YES  
G_RWA_00116     YES  
G_RWA_00258     YES  
G_RWA_00551     YES  
G_RWA_00901     YES

In [93]:
def highlight_diff(data, color='lightcoral'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Facebook', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

#df_final.style.apply(highlight_diff, axis=None)

In [ ]:
# coverage comparison report

In [117]:
# Facebook&Ericsson
cross_profile_fb_ec = datacompy.Compare(
RW_fc,
RW_ec,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='Ericsson'
)
# Facebook&ITU
cross_profile_fb_itu = datacompy.Compare(
RW_fc,
RW_it,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='ITU'
)
#Ericsson&ITU
cross_profile_Ericsson_itu = datacompy.Compare(
RW_ec,
RW_it,
join_columns='giga_school_id',  
df1_name='Ericsson', 
df2_name='ITU'
)

print(cross_profile_fb_ec.report())
print(cross_profile_fb_itu.report())
print(cross_profile_Ericsson_itu.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0  Facebook        6  4195
1  Ericsson        6  4197

Column Summary
--------------

Number of columns in common: 6
Number of columns in Facebook but not in Ericsson: 0
Number of columns in Ericsson but not in Facebook: 0

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 4,195
Number of rows in Facebook but not in Ericsson: 0
Number of rows in Ericsson but not in Facebook: 2

Number of rows with some compared columns unequal: 2,504
Number of rows with all compared columns equal: 1,691

Column Comparison
-----------------

Number of columns compared with some values unequal: 5
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 5,948

Columns with Unequal Values or Types
------------------------------------

          Column Face

In [94]:
# merging and selecting the most frequent values across the sources on each row and colunm
# creating a master coverage dataset and merging with the master school location dataset

In [102]:
#Main
df_c = pd.concat([RW_fc,RW_ec,RW_it])
RW_coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x:x.value_counts().index[0])
RW_master = pd.read_csv('./data/rwanda_school_geolocation_master.csv',low_memory=False,encoding='utf-8')
RW_master_w_coverage = RW_master.merge(RW_coverage, how="left", on=["giga_school_id"])
RW_master_w_coverage.to_csv('./data/RW_master_w_coverage.csv', encoding='utf-8')
RW_master_w_coverage_connectivity = RW_master_w_coverage.copy()
RW_master_w_coverage_connectivity['connectivity'] = np.where((RW_master_w_coverage_connectivity['connectivity_type'] != 'None'), 'YES', 'NO')
RW_master_w_coverage_connectivity.to_csv('./data/RW_master_w_coverage_connectivity.csv', encoding='utf-8')

In [103]:
RW_master_w_coverage.head()

,uuid,country,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,hex8,...,internet_availability,number_of_students,number_of_teachers,computers,connectivity_type,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,cdf4f80c-e3cc-4ebc-8931-8fe76018cc61,Rwanda,government,G_RWA_00002,521312.0,GS MUHORORO,Secondary,-2.159389,29.547972,886ade34e7fffff,...,USB_MODEM,28,1,260.0,3G,YES,4G,YES,NO,YES
1,a58be9dc-59c4-4cf9-96f5-e5b8fa84ba82,Rwanda,government,G_RWA_00005,331220.0,GS MWENDO,Secondary,-1.730639,30.212722,886ad88707fffff,...,USB_MODEM,62,3,272.0,3G,YES,4G,YES,YES,YES
2,9fe38ae7-7d51-4646-841b-1b6daa31c987,Rwanda,government,G_RWA_00006,411305.0,GS NYAGISOZI,Secondary,-2.295028,29.645556,886ade2481fffff,...,USB_MODEM,64,4,218.0,3G,YES,4G,YES,NO,YES
3,8b5c3930-c3ea-44d8-a510-086f8c6889aa,Rwanda,government,G_RWA_00009,510619.0,GS RUGANGO,Secondary,-2.371833,30.599167,886ad821d7fffff,...,USB_MODEM,80,2,252.0,3G,YES,None,YES,YES,NO
4,2288686a-e13e-423a-b0bb-b7d08452e9a8,Rwanda,government,G_RWA_00012,130610.0,GS GIHINGA,Secondary,-1.926806,30.591806,886ad8a095fffff,...,USB_MODEM,98,3,259.0,3G,YES,4G,YES,YES,YES


In [104]:
RW_master_w_coverage_connectivity.head()

,uuid,country,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,hex8,...,number_of_students,number_of_teachers,computers,connectivity_type,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage,connectivity
0,cdf4f80c-e3cc-4ebc-8931-8fe76018cc61,Rwanda,government,G_RWA_00002,521312.0,GS MUHORORO,Secondary,-2.159389,29.547972,886ade34e7fffff,...,28,1,260.0,3G,YES,4G,YES,NO,YES,YES
1,a58be9dc-59c4-4cf9-96f5-e5b8fa84ba82,Rwanda,government,G_RWA_00005,331220.0,GS MWENDO,Secondary,-1.730639,30.212722,886ad88707fffff,...,62,3,272.0,3G,YES,4G,YES,YES,YES,YES
2,9fe38ae7-7d51-4646-841b-1b6daa31c987,Rwanda,government,G_RWA_00006,411305.0,GS NYAGISOZI,Secondary,-2.295028,29.645556,886ade2481fffff,...,64,4,218.0,3G,YES,4G,YES,NO,YES,YES
3,8b5c3930-c3ea-44d8-a510-086f8c6889aa,Rwanda,government,G_RWA_00009,510619.0,GS RUGANGO,Secondary,-2.371833,30.599167,886ad821d7fffff,...,80,2,252.0,3G,YES,None,YES,YES,NO,YES
4,2288686a-e13e-423a-b0bb-b7d08452e9a8,Rwanda,government,G_RWA_00012,130610.0,GS GIHINGA,Secondary,-1.926806,30.591806,886ad8a095fffff,...,98,3,259.0,3G,YES,4G,YES,YES,YES,YES
